<h1><center>CSCI 3155 Principles of Programming Languages</center></h1>
<h2><center>Spring 2025</center></h2>

# Function Calls in Lettuce

Previously, we did not handle function calls in our fledgling Lettuce interpreter.  Now we will add support for function calls.  Before we do so, let us recall the full grammar.



$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & True \\
 & | & False \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\[5pt]
\end{array}$$




In [2]:
sealed trait Program
sealed trait Expr

case class TopLevel(e: Expr) extends Program

case class Const(v: Double) extends Expr // Expr -> Const(v)
case object True extends Expr // Expr -> True
case object False extends Expr // Expr -> False
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)
case class Mult(e1: Expr, e2: Expr) extends Expr // Expr -> Mult (Expr, Expr)

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

//Function definition
case class FunDef(param: String, bodyExpr: Expr) extends Expr

// Function call
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr

defined trait Program
defined trait Expr
defined class TopLevel
defined class Const
defined object True
defined object False
defined class Ident
defined class Plus
defined class Mult
defined class Geq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall

### Example 1

~~~
let square = function(x) 
           x * x in 
    square(10) 
~~~

This program evaluates to 100

### Example 2
~~~
let x = 10 in 
let y = 15 in 
let sq1 = function (x) 
            function (y) 
               x + y * y
           in

     (sq1(x))(y)
~~~
          
__Question:__ Map the different usages of x, y in the code above to the appropriate definitions?

This program evaluates to 73.

### Example 3
~~~
let f = function (x) 
            x + 2 
        in
let g = function (y)
            f(y) + y
        in
g(5)
~~~

### Example 4a

~~~
let z = 15 in
let g = z + 5 in
g
~~~

### Example 4b

~~~
let z = 15 in

let g = function (y)
            z + y
        in
g(5)
~~~


### Example 5

~~~
let z = 15 in
let foo = function (y)
            z + y
        in
let z = 20 in
foo (5)
~~~

### Example 6
~~~
let x = 10 in 
let y = 15 in 
let sq1 = function (x) 
            function (y) 
               x + y * y
           in
let sq2 = sq1(x) in
  sq2(y)
~~~

Evaluates to 235

### Example 7
~~~
let x = 10 in 
let y = 15 in 
let z = 20 in
let sq1 = function (x) 
            function (y) 
               x + y * z
           in
let sq2 = sq1(x) in 
let x = 8 in
let y = 6 in 
let z = 7 in
sq2(y)
~~~

Evaluates to 130

In [7]:
val z = 15
def foo(y:Int) = z+y
{
  val z = 20
  foo(5)
}


z: Int = 15
defined function foo
res6_2: Int = 20

In [6]:
val z = 15
def g(y:Int) = {
    z + y
}
g(5)



z: Int = 15
defined function g
res5_2: Int = 20

In [3]:
val p1 = TopLevel( 
    Let("square",                                // let square = 
         FunDef("x", Mult(Ident("x"), Ident("x"))),  //    function (x) x * x
         FunCall(Ident("square"), Const(10)) //     in  square(10)
       )
)

val x = Ident("x")
val y = Ident("y")
val fdef_inner = FunDef("y", Plus(x, Mult(y, y)))
val fdef_outer = FunDef("x", fdef_inner)
val call_expr = FunCall(FunCall(Ident("sq1"), x), y)
val sq1_call = Let("sq1", fdef_outer, call_expr)
val lety = Let("y", Const(15), sq1_call)
val letx = Let("x", Const(10), lety)
val p2 = TopLevel(letx)

p1: TopLevel = TopLevel(
  e = Let(
    s = "square",
    defExpr = FunDef(
      param = "x",
      bodyExpr = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "x"))
    ),
    bodyExpr = FunCall(
      funCalled = Ident(s = "square"),
      argExpr = Const(v = 10.0)
    )
  )
)
x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
fdef_inner: FunDef = FunDef(
  param = "y",
  bodyExpr = Plus(
    e1 = Ident(s = "x"),
    e2 = Mult(e1 = Ident(s = "y"), e2 = Ident(s = "y"))
  )
)
fdef_outer: FunDef = FunDef(
  param = "x",
  bodyExpr = FunDef(
    param = "y",
    bodyExpr = Plus(
      e1 = Ident(s = "x"),
      e2 = Mult(e1 = Ident(s = "y"), e2 = Ident(s = "y"))
    )
  )
)
call_expr: FunCall = FunCall(
  funCalled = FunCall(funCalled = Ident(s = "sq1"), argExpr = Ident(s = "x")),
  argExpr = Ident(s = "y")
)
sq1_call: Let = Let(
  s = "sq1",
  defExpr = FunDef(
    param = "x",
    bodyExpr = FunDef(
      param = "y",
      bodyExpr = Plus(
        e1 = Ident(s = "x"),
        e2 = Mult(e1 =

## Scoping for Function Calls: Static vs. Dynamic

It is important to understand how function calls capture scopes by considering a few examples both in lettuce and scala. 

### Example 1
~~~
let x = 10 in 
   let f = function(y) y * x in 
      let x = 15 in 
          f(10)
~~~

Or equivalently in scala

~~~
val x = 10
val f = (y: Int) => (y * x) 
{
   val x = 15 
      f(10)
}

~~~

In both cases, our code has a function `f` that multiplies its parameter `y` by `x`. But precisely which of the
`x` should the function use?

  - `let x = 10` in the first line that is in scope when the function was first defined?, or 
  - `let x = 15` in the third line that is in scope when the function is actually being called?


What would the Scala compiler do?